# Анализ работы модели предсказания качества воздуха

В работе используются:
- Модель Air Quality Prediction ([Kaggle](https://www.kaggle.com/code/hamedetezadi/air-quality-prediction)), в формате [ноутбука](https://colab.research.google.com/drive/14Tlqrmg76i7S33Gc5P5iFvjh7Ml2dXxB?usp=sharing)
- Датасет  «СОСТОЯНИЕ ЗАГРЯЗНЕНИЯ АТМОСФЕРЫ В
 ГОРОДАХ РОССИИ C 2007 ГОДА» - взят с сайта [Если быть точным](https://tochno.st/datasets/air_cities)


# 1. Скачивание датасета

In [79]:
import os

import kagglehub

source_dataset_name = "shrutibhargava94/india-air-quality-data"
source_dataset_files_path = kagglehub.dataset_download(source_dataset_name)

print(f"Скачиваем исходный датасет {source_dataset_name}")
for filename in os.listdir(source_dataset_files_path):
    source = os.path.join(source_dataset_files_path, filename)
    if os.path.isfile(source):
        with open(source, 'rb') as src:
            content = src.read()

        with open(os.path.join("./data", filename), 'w') as dst:
            dst.write(content.decode('utf-8', errors='ignore'))

print(f"Исходный датасет {source_dataset_name} помещен в папку проекта data")

Скачиваем исходный датасет shrutibhargava94/india-air-quality-data
Исходный датасет shrutibhargava94/india-air-quality-data помещен в папку проекта data


## 2. Импорт необходимых зависимостей

In [80]:
import pandas as pd
import numpy as np
import joblib as jl

## 3. Описание используемого датасета

Данные датасета приведены в соответствии с теми, что были использованы при построении исходной модели с kaggle.
* Поле type (тип местности, где был произведен замер) заполнено значением "Промышленные предприятия", т.к. все данные собраны по ним.
* Поле spm (взвешенные твердые частицы) заполненно значениями поля solid emissions исходного сета.
* Поле rspm (концентрация вдыхаемых взвешенных частиц) - значение поля spm умноженного на коэффциент 0,6 (выбран эмпирически).
* Поле pm2_5 (твердые частицы) не заполнено, в расчетах исходной модели роли не играет.

In [81]:
# данные датасета приведенные к требуемому виду для использования исходной моделью
data = pd.read_csv('data/upd_data_air_cities_100_v20231129_data_air_cities_100_v20231129.csv')
data.head()

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,1701000,2007,Алтайский край,БАРНАУЛ,NaN,Промышленные предприятия,14.7,4.3,14.28,23.8,NaN,NaN,2007
1,1705000,2007,Алтайский край,БИЙСК,NaN,Промышленные предприятия,7.4,2.9,5.40,9.0,NaN,NaN,2007
2,1706000,2007,Алтайский край,ЗАРИНСК,NaN,Промышленные предприятия,4.8,2.4,1.74,2.9,NaN,NaN,2007
3,10701000,2007,Амурская область,БЛАГОВЕЩЕНСК,NaN,Промышленные предприятия,3.8,4.0,7.38,12.3,NaN,NaN,2007
4,10712000,2007,Амурская область,ЗЕЯ,NaN,Промышленные предприятия,0.3,0.2,0.24,0.4,NaN,NaN,2007


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3992 entries, 0 to 3991
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   stn_code                     3992 non-null   int64  
 1   sampling_date                3992 non-null   int64  
 2   state                        3992 non-null   object 
 3   location                     3992 non-null   object 
 4   agency                       0 non-null      float64
 5   type                         3992 non-null   object 
 6   so2                          3991 non-null   float64
 7   no2                          3991 non-null   float64
 8   rspm                         3992 non-null   float64
 9   spm                          3991 non-null   float64
 10  location_monitoring_station  0 non-null      float64
 11  pm2_5                        0 non-null      float64
 12  date                         3992 non-null   int64  
dtypes: float64(7), int

## 4. Очистка исходных данных

In [82]:
data.replace([-1, -0.6], np.nan, inplace=True)

## 4. Подготовка существующей модели

In [83]:
# Загрузка существующей модели
model_name = "Air_Quality_Prediction.joblib"
model_path = f"model/{model_name}"
model = jl.load(model_path)

# Определяем ожидаемые признаки для модели
expected_features = model.feature_names_in_
print("Ожидаемые моделью признаки:", expected_features)
print("Текущие признаки:", data.columns.tolist())

def calculate_soi(so2):
    si = 0
    if so2 <= 40:
        si = so2 * (50 / 40)
    elif 40 < so2 <= 80:
        si = 50 + (so2 - 40) * (50 / 40)
    elif 80 < so2 <= 380:
        si = 100 + (so2 - 80) * (100 / 300)
    elif 380 < so2 <= 800:
        si = 200 + (so2 - 380) * (100 / 420)
    elif 800 < so2 <= 1600:
        si = 300 + (so2 - 800) * (100 / 800)
    elif so2 > 1600:
        si = 400 + (so2 - 1600) * (100 / 800)

    return si


def calculate_noi(no2):
    if no2 <= 40:
        ni = no2 * 50 / 40
    elif 40 < no2 <= 80:
        ni = 50 + (no2 - 40) * (50 / 40)
    elif 80 < no2 <= 180:
        ni = 100 + (no2 - 80) * (100 / 100)
    elif 180 < no2 <= 280:
        ni = 200 + (no2 - 180) * (100 / 100)
    elif 280 < no2 <= 400:
        ni = 300 + (no2 - 280) * (100 / 120)
    else:
        ni = 400 + (no2 - 400) * (100 / 120)

    return ni


def calculate_rpi(rspm):
    rpi = 0
    if rspm <= 30:
        rpi = rspm * 50 / 30
    elif 30 < rspm <= 60:
        rpi = 50 + (rspm - 30) * 50 / 30
    elif 60 < rspm <= 90:
        rpi = 100 + (rspm - 60) * 100 / 30
    elif 90 < rpi <= 120:
        rpi = 200 + (rspm - 90) * 100 / 30
    elif 120 < rpi <= 250:
        rpi = 300 + (rspm - 120) * (100 / 130)
    else:
        rpi = 400 + (rspm - 250) * (100 / 130)

    return rpi


def calculate_spi(spm):
    if spm <= 50:
        spi = spm * 50 / 50
    elif 50 < spm <= 100:
        spi = 50 + (spm - 50) * (50 / 50)
    elif 100 < spm <= 250:
        spi = 100 + (spm - 100) * (100 / 150)
    elif 250 < spm <= 350:
        spi = 200 + (spm - 250) * (100 / 100)
    elif 350 < spm <= 430:
        spi = 300 + (spm - 350) * (100 / 80)
    else:
        spi = 400 + (spm - 430) * (100 / 430)

    return spi

# Добавляем расчетные значения 'SOi' 'Noi' 'Rpi' 'SPMi'
data['SOi'] = data['so2'].apply(calculate_soi)
data['Noi'] = data['no2'].apply(calculate_noi)
data['Rpi'] = data['rspm'].apply(calculate_rpi)
data['SPMi'] = data['spm'].apply(calculate_spi)

# Подготавливаем список отсутствующих в наших данных признаков
missing_features = set(expected_features) - set(data.columns)
print("Отсутствующие признаки:", missing_features) if missing_features else print(
    "Отсутствующих для работы модели признаков нет")

# Добавляем отсутствующие признаки, заполняя их нулями
for feature in missing_features:
    data[feature] = 0

# Оставляем только нужные признаки для модели
data_final = data[list(expected_features)].copy()
# TODO убираю все NaN, не надо так, но думать некогда
data_final.dropna(inplace=True)
data_final.head()

Ожидаемые моделью признаки: ['SOi' 'Noi' 'Rpi' 'SPMi']
Текущие признаки: ['stn_code', 'sampling_date', 'state', 'location', 'agency', 'type', 'so2', 'no2', 'rspm', 'spm', 'location_monitoring_station', 'pm2_5', 'date']
Отсутствующих для работы модели признаков нет


,SOi,Noi,Rpi,SPMi
0,18.375,5.375,23.8,23.8
1,9.250,3.625,9.0,9.0
2,6.000,3.000,2.9,2.9
3,4.750,5.000,12.3,12.3
4,0.375,0.250,0.4,0.4


## 5 Использование модели для прогнозирования

In [84]:
# Прогнозирование
predictions = model.predict(data_final)
data_final['Predictions'] = predictions

# Сохранение обработанных данных
processed_file_path = 'data/processed_data_air_quality.csv'
data_final.to_csv(processed_file_path, index=False)
print(f"Обработанные данные сохранены в {processed_file_path}")

Обработанные данные сохранены в data/processed_data_air_quality.csv
